In [1]:
# libraries 
import pandas as pd 
import os
import numpy as np
import time
import random
from csv import reader
import time

# plots:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# scraping
from bs4 import BeautifulSoup
from lxml import etree #for using XPATH with beautifulsoup
import requests
import numpy as np

#JSON
import json

# GeoJSON
# import geopandas as gpd

# regular expression
import re 

# concurrent futures - boosts the process of scraping utilicing the CPU better
# two main classes- 
#       1) executor class: manages all the threads and workload
#       2) futures class: creates a little instance and manages data coming back
import concurrent.futures

# libraries: 
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import glob 

# Setting up User agent and header 


In [2]:
# change user agent for each request randomly
def get_user_agent():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.57'
    ]
    user_agent = user_agent_list[random.randint(0,len(user_agent_list)-1)]
   
    return user_agent

def get_header():
    
    # get user-agent
    user_agent = get_user_agent()

     # set up header     
    header = {
        'authority': 'api.boligsiden.dk',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'origin': 'https://www.boligsiden.dk',
        'referer': 'https://www.boliga.dk/',
        'sec-ch-ua': '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': user_agent
#         'x-api-key': 'GWD0fljZzkc8GOLV',
    }
    return header 

# scrape page 

In [3]:
################################## for years 2006 to 2011

def scrape_property(guid):
    global dataframes_list
    global counter_property
    global session 
    
    

    # Set up the header
    ########################
    main_header = get_header()

    # Set up the url of the API
    ###########################
    
    params = {
        'id': guid,
    }

    #     main_url = f'https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&propertyType=1,2,3,6&saleType=1&salesDateMin=2006&salesDateMax=2011&sort=date-d&page={page}'

    # Make the HTTPS Request 
    ########################
    response = session.get('https://api.boliga.dk/api/v2/bbrinfo/bbr', params=params, headers=main_header)
#     response = session.get(main_url, headers=main_header)

    if response.status_code == 200:
        
#         print('--------------------- scraping !')

        # Convert to JSON 
        #################
        page_result = response.json()

        # construct the dataframe from the json file
        ####################################################
        dataframes = pd.DataFrame([page_result])

        # append to the dataframes_list 
        ###############################
        dataframes_list.append(dataframes)
    else:
        print()
        print('############################################################')
        print('------------------- BAD REQUEST, NOT 200 ------------------')
        print('############################################################')
        print()

    
    

# main function

In [4]:
##################################
# Get the Guid values as a list 
# ---- the GUID values are the only key for the api to get the property attributes 
##################################
# Specify the path to your CSV file
path = r"D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_2_merge_the csv_chunks_propertyIDlinks_filter_out_duplicates\Boliga_property_ID_links_unique_at_guid.csv"


# Specify the name of the column you want to read
column_name = "guid"

# Read the specific column from the CSV file
guid_data = pd.read_csv(path, usecols=[column_name])

# convert to list 
guid_data = guid_data['guid'].tolist()


# set up a session to speed things up
session = requests.Session()

# start scraping : 
counter = 0
counter_property = 0

chunk_size = 10000

for i in range(0,len(guid_data),chunk_size):
    counter +=1
    print('-------------------------------------')
    print(f'---- Chunk number {counter} of 104 chunks (1.031.607 properties / 10.000)')
    print('-------------------------------------')
    
    # create the database
    dataframes_list = []
    
    # get the guid chunk
    chunk = guid_data[i:i + chunk_size]

    # --------------------------- for loop version ---------------
#     for guid in chunk:
#         scrape_property(guid)
    # --------------------------- Concurrent futures version ---------------
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor: 
        retry = Retry(connect=3, backoff_factor=0.5) # max 3 retries to the same link with a 0.5 sec delay
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        # run script
        executor.map(scrape_property,chunk)
    
    
    # merge all the frames into a single dataframe
    combined_df = pd.concat(dataframes_list, ignore_index=True)
    
    # Save each chunk to a CSV file with a unique name
    csv_filename = f'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_3_Scrape_The_properties\Boliga_property_scrape_chunk_{counter}.csv'
    combined_df.to_csv(csv_filename, index=False, encoding='utf-8')
    print('--------------------- chunks saved to csv file !')
       


-------------------------------------
---- Chunk number 1 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 2 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 3 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 4 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 5 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Ch

--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 44 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 45 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 46 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 47 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to csv file !
-------------------------------------
---- Chunk number 48 of 104 chunks (1.031.607 properties / 10.000)
-------------------------------------
--------------------- chunks saved to cs


############################################################
------------------- BAD REQUEST, NOT 200 ------------------
############################################################


############################################################
------------------- BAD REQUEST, NOT 200 ------------------
############################################################


############################################################
------------------- BAD REQUEST, NOT 200 ------------------
############################################################


############################################################
------------------- BAD REQUEST, NOT 200 ------------------
############################################################


############################################################
------------------- BAD REQUEST, NOT 200 ------------------
############################################################


############################################################
------------------

In [8]:
input_folder = f'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_3_Scrape_The_properties'
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

nr_properties_rows_total = 0
nr_properties_rows_list = []
nr_column = []
counter=0

for csv_file in csv_files:
    counter +=1
    
    print(f'########################## file nr. {counter} out of 104 ######################')
    
    #get full path
    file_path = os.path.join(input_folder, csv_file)
    
    # read in the csv file
    df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)
    
    print('**data on csv file :**')
    
    # computing number of columns
    cols = len(df.axes[1])
    print(f'----------------- Number of columns csv file: {cols}')
    print()
    nr_column.append(cols)
    
    # computing number of rows
    rows = len(df.axes[0])
    print(f'----------------- Number of rows csv file: {rows}')
    nr_properties_rows_total+=rows
    nr_properties_rows_list.append(rows)

print(f'total number of rows: {nr_properties_rows_total}')

########################## file nr. 1 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 2 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 3 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 4 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 5 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## fil

**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 44 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 45 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 46 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 47 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 48 out of 104 ######################
**data on csv file :**
--

**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 9988
########################## file nr. 87 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 88 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 89 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 90 out of 104 ######################
**data on csv file :**
----------------- Number of columns csv file: 48

----------------- Number of rows csv file: 10000
########################## file nr. 91 out of 104 ######################
**data on csv file :**
---

In [10]:
nr_column

[48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48]

In [1]:
1031607/10000

103.1607

In [ ]:
chunk_size = 3

bla = [1,2,3,4,5,6,7,8,9,10,12,13,14,15]

for i in range(0,len(bla),chunk_size):
    
    listi = bla[i:i+chunk_size]
    for item in listi:
        print(item)
    print()


In [ ]:
property_code = '9133416F-191B-496E-88BB-62B3A46A370A'

# Set up the header
########################
main_header = get_header()

# Set up the url of the API
###########################

params = {
    'id': property_code,
}


# Make the HTTPS Request 
########################
response = requests.get('https://api.boliga.dk/api/v2/bbrinfo/bbr', params=params, headers=main_header)
#     response = session.get(main_url, headers=main_header)

if response.status_code == 200:

    print('--------------------- scraping !')

    # Convert to JSON 
    #################
    page_result = response.json()

    

In [ ]:
dataframes = pd.DataFrame([page_result])
dataframes

In [ ]:
dataframes_list = []

# construct the dataframe from the json file
####################################################
dataframes = pd.DataFrame([page_result])

# append to the dataframes_list 
###############################
dataframes_list.append(dataframes)


In [ ]:
dataframes_list[0]